[Reference](https://medium.com/@ryan_forrester_/python-multiple-exception-handling-a-complete-guide-088f0f70eee4)

# Understanding Multiple Exception Handling

In [1]:
def divide_numbers(a, b):
    try:
        result = a / b
        # Convert result to integer
        return int(result)
    except ZeroDivisionError:
        print("Error: Division by zero!")
        return None
    except ValueError:
        print("Error: Couldn't convert to integer!")
        return None

# Example usage:
print(divide_numbers(10, 2))    # Output: 5
print(divide_numbers(10, 0))    # Output: Error: Division by zero! None
print(divide_numbers(10.5, 2))  # Output: 5

5
Error: Division by zero!
None
5


# Catching Multiple Exceptions in One Line

In [2]:
def process_data(data):
    try:
        # Try to convert data to float and process it
        value = float(data)
        result = 100 / value
        return result
    except (ValueError, ZeroDivisionError) as e:
        # Handle both conversion errors and division by zero
        print(f"Error processing data: {str(e)}")
        return None
    except TypeError as e:
        print(f"Invalid data type: {str(e)}")
        return None

# Example usage:
print(process_data("10"))      # Output: 10.0
print(process_data("zero"))    # Output: Error processing data: could not convert...
print(process_data("0"))       # Output: Error processing data: division by zero
print(process_data(None))      # Output: Invalid data type: float() argument...

10.0
Error processing data: could not convert string to float: 'zero'
None
Error processing data: float division by zero
None
Invalid data type: float() argument must be a string or a real number, not 'NoneType'
None


# Using Exception Hierarchies

In [3]:
def read_configuration(filename):
    try:
        with open(filename) as f:
            data = f.read()
            config = eval(data)  # Don't do this in real code! Used for example only
            return config
    except FileNotFoundError:
        # Specific: Handle missing file
        print(f"Config file '{filename}' not found")
        return {}
    except OSError as e:
        # Parent: Handle other OS-related errors
        print(f"OS error occurred: {e}")
        return {}
    except Exception as e:
        # Catch-all: Handle unexpected errors
        print(f"Unexpected error: {e}")
        return {}

# Example usage:
config = read_configuration("nonexistent.conf")  # Output: Config file 'nonexistent.conf' not found

Config file 'nonexistent.conf' not found


# Adding else and finally

In [5]:
def update_user_preferences(user_id, preferences):
    db_connection = None
    try:
        db_connection = connect_to_database()  # Hypothetical function
        current_prefs = get_preferences(user_id)
        current_prefs.update(preferences)
        save_preferences(user_id, current_prefs)
    except ConnectionError:
        print("Database connection failed")
        return False
    except KeyError:
        print("Invalid user ID")
        return False
    else:
        # Runs only if no exception occurred
        print("Preferences updated successfully")
        return True
    finally:
        # Always runs, whether exception occurred or not
        if db_connection:
            db_connection.close()

# Example usage:
success = update_user_preferences(123, {"theme": "dark"})

# Creating Custom Exception Hierarchies

In [6]:
class DataProcessingError(Exception):
    """Base class for data processing exceptions"""
    pass

class DataFormatError(DataProcessingError):
    """Raised when data format is invalid"""
    pass

class DataValidationError(DataProcessingError):
    """Raised when data validation fails"""
    pass

def process_user_data(data):
    try:
        if not isinstance(data, dict):
            raise DataFormatError("Data must be a dictionary")

        if "age" not in data:
            raise DataValidationError("Age is required")

        if not isinstance(data["age"], int):
            raise DataFormatError("Age must be an integer")

        if data["age"] < 0:
            raise DataValidationError("Age cannot be negative")

    except DataFormatError as e:
        print(f"Format error: {e}")
        return False
    except DataValidationError as e:
        print(f"Validation error: {e}")
        return False
    except Exception as e:
        print(f"Unexpected error: {e}")
        return False
    else:
        print("Data processed successfully")
        return True

# Example usage:
data1 = {"name": "John", "age": "25"}  # Wrong format
data2 = {"name": "John", "age": -5}    # Invalid value
data3 = {"name": "John", "age": 25}    # Correct

print(process_user_data(data1))  # Output: Format error: Age must be an integer
print(process_user_data(data2))  # Output: Validation error: Age cannot be negative
print(process_user_data(data3))  # Output: Data processed successfully

Format error: Age must be an integer
False
Validation error: Age cannot be negative
False
Data processed successfully
True


In [7]:
import json
import csv
from pathlib import Path

def convert_json_to_csv(input_path, output_path):
    try:
        # Check if input file exists
        if not Path(input_path).exists():
            raise FileNotFoundError(f"Input file not found: {input_path}")

        # Read JSON data
        with open(input_path, 'r') as json_file:
            try:
                data = json.load(json_file)
            except json.JSONDecodeError as e:
                raise DataFormatError(f"Invalid JSON format: {str(e)}")

        # Ensure data is a list of dictionaries
        if not isinstance(data, list):
            raise DataFormatError("JSON data must be a list of records")

        if not data:
            raise DataValidationError("JSON data is empty")

        # Get field names from first record
        fieldnames = data[0].keys()

        # Write CSV file
        with open(output_path, 'w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(data)

    except (FileNotFoundError, DataFormatError, DataValidationError) as e:
        print(f"Error: {str(e)}")
        return False
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return False
    else:
        print(f"Successfully converted {input_path} to {output_path}")
        return True

# Example usage:
input_file = "data.json"
output_file = "output.csv"
success = convert_json_to_csv(input_file, output_file)

Error: Input file not found: data.json


In [8]:
class DatabaseConnection:
    def __init__(self, connection_string):
        self.connection_string = connection_string
        self.connection = None

    def __enter__(self):
        try:
            # Simulate database connection
            print(f"Connecting to database: {self.connection_string}")
            self.connection = True
            return self
        except Exception as e:
            raise ConnectionError(f"Failed to connect: {str(e)}")

    def __exit__(self, exc_type, exc_val, exc_tb):
        # This runs even if an exception occurs
        if self.connection:
            print("Closing database connection")
            self.connection = None
        # Return False to propagate exceptions, True to suppress them
        return False

def process_user_records(users):
    with DatabaseConnection("postgresql://localhost:5432/users") as db:
        try:
            for user in users:
                print(f"Processing user: {user}")
                # Simulate some database operations
                if not isinstance(user, dict):
                    raise TypeError("User must be a dictionary")
        except TypeError as e:
            print(f"Invalid user data: {e}")
            return False
    return True

# Example usage:
users = [{"id": 1, "name": "Alice"}, {"id": 2, "name": "Bob"}, "invalid"]
process_user_records(users)

Connecting to database: postgresql://localhost:5432/users
Processing user: {'id': 1, 'name': 'Alice'}
Processing user: {'id': 2, 'name': 'Bob'}
Processing user: invalid
Invalid user data: User must be a dictionary
Closing database connection


False

In [9]:
class ConfigurationError(Exception):
    pass

def load_config(filename):
    try:
        with open(filename) as f:
            config_text = f.read()
            try:
                return json.loads(config_text)
            except json.JSONDecodeError as e:
                # Raise new exception but keep original error info
                raise ConfigurationError("Invalid configuration format") from e
    except FileNotFoundError as e:
        raise ConfigurationError("Configuration file missing") from e

def initialize_application():
    try:
        config = load_config("config.json")
        print("Application initialized")
    except ConfigurationError as e:
        print(f"Failed to initialize: {e}")
        # Access original exception if it exists
        if e.__cause__:
            print(f"Original error: {e.__cause__}")

# Example usage:
initialize_application()

Failed to initialize: Configuration file missing
Original error: [Errno 2] No such file or directory: 'config.json'


In [10]:
import asyncio
from typing import List

async def fetch_user(user_id: int) -> dict:
    # Simulate API call
    await asyncio.sleep(1)
    if user_id < 0:
        raise ValueError("Invalid user ID")
    return {"id": user_id, "name": f"User {user_id}"}

async def process_users(user_ids: List[int]):
    tasks = []
    results = []
    errors = []

    # Create tasks for all users
    for user_id in user_ids:
        task = asyncio.create_task(fetch_user(user_id))
        tasks.append(task)

    # Wait for all tasks to complete
    for task in tasks:
        try:
            result = await task
            results.append(result)
        except ValueError as e:
            errors.append(f"Value error: {str(e)}")
        except Exception as e:
            errors.append(f"Unexpected error: {str(e)}")

    return results, errors

# Example usage:
async def main():
    user_ids = [1, 2, -1, 4]
    results, errors = await process_users(user_ids)
    print("Results:", results)
    print("Errors:", errors)

# Run the async code
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [11]:
from functools import wraps
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def handle_exceptions(retries=3, allowed_exceptions=(ConnectionError,)):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < retries:
                try:
                    return func(*args, **kwargs)
                except allowed_exceptions as e:
                    attempts += 1
                    if attempts == retries:
                        logger.error(f"Failed after {retries} attempts: {str(e)}")
                        raise
                    logger.warning(f"Attempt {attempts} failed, retrying...")
                except Exception as e:
                    logger.error(f"Unexpected error: {str(e)}")
                    raise
            return None
        return wrapper
    return decorator

@handle_exceptions(retries=2)
def fetch_data(url):
    # Simulate network request that might fail
    if "invalid" in url:
        raise ConnectionError("Failed to connect")
    return f"Data from {url}"

# Example usage:
try:
    data = fetch_data("https://invalid.example.com")
except ConnectionError:
    print("Failed to fetch data")

ERROR:__main__:Failed after 2 attempts: Failed to connect


Failed to fetch data


In [12]:
import threading
from queue import Queue
import traceback

class WorkerThread(threading.Thread):
    def __init__(self, task_queue: Queue, error_queue: Queue):
        super().__init__()
        self.task_queue = task_queue
        self.error_queue = error_queue

    def run(self):
        while True:
            try:
                task = self.task_queue.get()
                if task is None:  # Poison pill
                    break

                # Process task
                result = self.process_task(task)
                print(f"Processed task: {result}")

            except Exception as e:
                # Capture full exception info
                error_info = {
                    'error': e,
                    'traceback': traceback.format_exc(),
                    'task': task
                }
                self.error_queue.put(error_info)
            finally:
                self.task_queue.task_done()

    def process_task(self, task):
        if task < 0:
            raise ValueError("Negative values not allowed")
        return task * 2

def run_worker_pool(tasks, num_workers=3):
    task_queue = Queue()
    error_queue = Queue()
    workers = []

    # Create worker threads
    for _ in range(num_workers):
        worker = WorkerThread(task_queue, error_queue)
        worker.start()
        workers.append(worker)

    # Add tasks to queue
    for task in tasks:
        task_queue.put(task)

    # Add poison pills to stop workers
    for _ in range(num_workers):
        task_queue.put(None)

    # Wait for all tasks to complete
    task_queue.join()

    # Check for errors
    errors = []
    while not error_queue.empty():
        errors.append(error_queue.get())

    return errors

# Example usage:
tasks = [1, 2, -3, 4, -5]
errors = run_worker_pool(tasks)
for error in errors:
    print(f"Error processing task {error['task']}: {error['error']}")

Processed task: 2
Processed task: 4
Processed task: 8
Error processing task -3: Negative values not allowed
Error processing task -5: Negative values not allowed
